# Carga Datos Vid Botritis.
## Fructificación (julio-septiembre).
## Riesgo aparición botritis (Nº días con Tmed > 15ºC y HR > 80%)
## Pablo Lavín

In [1]:
# Cargamos paquetes

library(repr)
library(dplyr)

library(abind)
library(loadeR)
library(transformeR)
library(visualizeR)
library(downscaleR)
library(climate4R.UDG)
library(climate4R.climdex)
library(climate4R.indices)
library(easyVerification)

library(lattice)
library(magrittr)
library(gridExtra)
library(RColorBrewer)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: rJava

Loading required package: loadeR.java

Java version 23x amd64 by N/A detected

NetCDF Java Library v4.6.0-SNAPSHOT (23 Apr 2015) loaded and ready

Loading required package: climate4R.UDG

climate4R.UDG version 0.2.6 (2023-06-26) is loaded

Please use 'citation("climate4R.UDG")' to cite this package.

loadeR version 1.8.1 (2023-06-22) is loaded


Get the latest stable version (1.8.2) using <devtools::install_github(c('SantanderMetGroup/climate4R.UDG','SantanderMetGroup/loadeR'))>

Please use 'citation("loadeR")' to cite this package.




    _______   ____  ___________________  __  ________ 
   / ___/ /  / /  |/  / __  /_  __/ __/ / / / / __  / 
  / /  / /  / / /|_/ / /_/ / / / / __/ / /_/ / /_/_/  
 / /__/ /__/ / /  / / __  / / / / /__ /___  / / \ \ 
 \___/____/_/_/  /_/_/ /_/ /_/  \___/    /_/\/   \_\ 
 
      github.com/SantanderMetGroup/climate4R



transformeR version 2.2.2 (2023-10-26) is loaded


Get the latest stable version (2.2.3) using <devtools::install_github('SantanderMetGroup/transformeR')>

Please see 'citation("transformeR")' to cite this package.

visualizeR version 1.6.4 (2023-10-26) is loaded

Please see 'citation("visualizeR")' to cite this package.

downscaleR version 3.3.4 (2023-06-22) is loaded

Please use 'citation("downscaleR")' to cite this package.

Loading required package: climdex.pcic

Loading required package: PCICt

climate4R.climdex version 0.2.3 (2023-06-23) is loaded

Use 'climdexShow()' for an overview of the available ETCCDI indices

climate4R.indices version 0.3.1 (2023-06-22) is loaded

Use 'indexShow()' for an overview of the available climate indices and circIndexShow() for the circulation indices.

NOTE: use package climate4R.climdex to calculate ETCCDI indices.


Attaching package: ‘climate4R.indices’


The following object is masked from ‘package:transformeR’:

    lambWT


Loading required

In [2]:
# Region de estudio

lon = c(-10, 20)
lat = c(35,46)

## Cargar hindcast (leadtime 0, 1, 2, 3)

In [3]:
anios = 1981:2016
meses_ini = c("07", "06", "05", "04")  # meses de inicialización

# Función generalizada para cargar los datos por mes y año
cargar_dato = function(anio, mes_ini) {

    yyyymm = paste0(anio, mes_ini)

    ruta = paste0(
        "/lustre/gmeteo/PTICLIMA/DATA/SEASONAL/",
        "seasonal-original-single-levels/medcof/hindcast/tas/ecmwf/51/", yyyymm, "/",
        "seasonal-original-single-levels_medcof_hindcast_tas_ecmwf_51_", yyyymm, ".ncml"
    )

    data_aux = loadGridData(dataset = ruta,
                            var = "tas",
                            lonLim = lon,
                            latLim = lat,
                            season = c(7, 8, 9)) %>% suppressMessages %>% suppressWarnings

    return(data_aux)
}

# Creo una lista donde cada elemento es la salida para un mes de inicialización diferente
resultados_por_mes = lapply(meses_ini, function(mes) {
    lapply(anios, function(anio) cargar_dato(anio, mes))
})

# Nombro los elementos por mes
names(resultados_por_mes) = paste0("mes_", meses_ini)

In [4]:
# Asigno nombre a cada grid con los leadtime
hindcast_0 = resultados_por_mes[["mes_07"]]
hindcast_1 = resultados_por_mes[["mes_06"]]
hindcast_2 = resultados_por_mes[["mes_05"]]
hindcast_3 = resultados_por_mes[["mes_04"]]

# Combinamos los grids en la dimensión temporal
hindcast_0_grid = bindGrid(hindcast_0, dimension = "time")
hindcast_1_grid = bindGrid(hindcast_1, dimension = "time")
hindcast_2_grid = bindGrid(hindcast_2, dimension = "time")
hindcast_3_grid = bindGrid(hindcast_3, dimension = "time")

## Cargar forecast (leadtime 0, 1, 2, 3)

In [5]:
anios = 2017:2022
meses_ini = c("07", "06", "05", "04")  # meses de inicialización

# Función generalizada para cargar los datos por mes y año
cargar_dato = function(anio, mes_ini) {

    yyyymm = paste0(anio, mes_ini)

    ruta = paste0(
        "/lustre/gmeteo/PTICLIMA/DATA/SEASONAL/",
        "seasonal-original-single-levels/medcof/forecast/tas/ecmwf/51/", yyyymm, "/",
        "seasonal-original-single-levels_medcof_forecast_tas_ecmwf_51_", yyyymm, ".ncml"
    )

    data_aux = loadGridData(dataset = ruta,
                            var = "tas",
                            lonLim = lon,
                            latLim = lat,
                            season = c(7, 8, 9)) %>% suppressMessages %>% suppressWarnings

    return(data_aux)
}

# Creo una lista donde cada elemento es la salida para un mes de inicialización diferente
forecast = lapply(meses_ini, function(mes) {
    lapply(anios, function(anio) cargar_dato(anio, mes))
})

# Nombro los elementos por mes
names(forecast) = paste0("mes_", meses_ini)

In [6]:
# Asigno nombre a cada grid con los leadtime
forecast_0 = forecast[["mes_07"]]
forecast_1 = forecast[["mes_06"]]
forecast_2 = forecast[["mes_05"]]
forecast_3 = forecast[["mes_04"]]

# Combinamos los grids en la dimensión temporal
forecast_0_grid = bindGrid(forecast_0, dimension = "time")
forecast_1_grid = bindGrid(forecast_1, dimension = "time")
forecast_2_grid = bindGrid(forecast_2, dimension = "time")
forecast_3_grid = bindGrid(forecast_3, dimension = "time")

# Me quedo con los primeros 25 miembros
forecast_0_members = subsetGrid(forecast_0_grid, members = 1:25)
forecast_1_members = subsetGrid(forecast_1_grid, members = 1:25)
forecast_2_members = subsetGrid(forecast_2_grid, members = 1:25)
forecast_3_members = subsetGrid(forecast_3_grid, members = 1:25)

# Combinamos los grids de hindcast y forecast
ecmwf_0_grid = bindGrid(hindcast_0_grid, forecast_0_members, dimension = "time")
ecmwf_1_grid = bindGrid(hindcast_1_grid, forecast_1_members, dimension = "time")
ecmwf_2_grid = bindGrid(hindcast_2_grid, forecast_2_members, dimension = "time")
ecmwf_3_grid = bindGrid(hindcast_3_grid, forecast_3_members, dimension = "time")

## Cargar las observaciones ERA5-Land (tas)

In [7]:
# Define años y meses
anios = 1981:2022
meses = sprintf("%02d", c(7, 8, 9))

# Función para construir la ruta y cargar los datos
cargar_dato = function(anio, mes) {
    yyyy = paste0(anio)
    yyyymm = paste0(anio, mes)
    ruta = paste0(
        "/lustre/gmeteo/PTICLIMA/DATA/REANALYSIS/ERA5-Land/data/Iberia/day/t2m/", yyyy, "/",
        "t2m_ERA5-Land_", yyyymm, ".nc"
    )
    
    # Carga el dataset
    data_aux = loadGridData(dataset = ruta,
                            var = "t2m",
                            lonLim = lon,
                            latLim = lat) %>% suppressMessages %>% suppressWarnings

    return(data_aux)
}

# Crear combinaciones
combinaciones = do.call(rbind, lapply(anios, function(anio) {
  data.frame(
    anio = c(anio, anio, anio),
    mes  = c("07", "08", "09"),
    stringsAsFactors = FALSE
  )
}))

# Aplicar la función a cada combinación
era5_data = lapply(1:nrow(combinaciones), function(i) {
    cargar_dato(combinaciones$anio[i], combinaciones$mes[i])
})

In [8]:
# Combinamos los grids en la dimensión temporal
era5_time = bindGrid(era5_data, dimension = "time")

# Upscaling de la resolución de las observaciones
era5_ups = interpGrid(era5_time,
                      new.coordinates = getGrid(ecmwf_0_grid),
                      method = "bilinear") %>% suppressMessages %>% suppressWarnings

In [9]:
# Recorto los datos a la región de España
modelo_0 = subsetGrid(ecmwf_0_grid, lonLim = c(-10, 5), latLim = c(35, 44))
modelo_1 = subsetGrid(ecmwf_1_grid, lonLim = c(-10, 5), latLim = c(35, 44))
modelo_2 = subsetGrid(ecmwf_2_grid, lonLim = c(-10, 5), latLim = c(35, 44))
modelo_3 = subsetGrid(ecmwf_3_grid, lonLim = c(-10, 5), latLim = c(35, 44))

obs = subsetGrid(era5_ups, lonLim = c(-10, 5), latLim = c(35, 44))

# Pasamos las observaciones de Kelvin a Celsius
obs_cel = gridArithmetics(obs, 273.15, operator = "-")

In [10]:
# Guardo los datos
saveRDS(modelo_0, file = "tas_botritis_model_vid_0.rds")
saveRDS(modelo_1, file = "tas_botritis_model_vid_1.rds")
saveRDS(modelo_2, file = "tas_botritis_model_vid_2.rds")
saveRDS(modelo_3, file = "tas_botritis_model_vid_3.rds")

saveRDS(obs_cel, file = "tas_botritis_obs_vid.rds")

## Cargar hindcast hr (leadtime 0, 1, 2, 3)

In [11]:
anios = 1981:2016
meses_ini = c("07", "06", "05", "04")  # meses de inicialización

# Función generalizada para cargar los datos por mes y año
cargar_dato = function(anio, mes_ini) {

    yyyymm = paste0(anio, mes_ini)

    ruta = paste0(
        "/lustre/gmeteo/PTICLIMA/DATA/SEASONAL/",
        "seasonal-original-single-levels/medcof/hindcast/hurs/ecmwf/51/", yyyymm, "/",
        "seasonal-original-single-levels_medcof_hindcast_hurs_ecmwf_51_", yyyymm, ".ncml"
    )

    data_aux = loadGridData(dataset = ruta,
                            var = "hurs",
                            lonLim = lon,
                            latLim = lat,
                            season = c(7, 8, 9)) %>% suppressMessages %>% suppressWarnings

    return(data_aux)
}

# Creo una lista donde cada elemento es la salida para un mes de inicialización diferente
resultados_por_mes = lapply(meses_ini, function(mes) {
    lapply(anios, function(anio) cargar_dato(anio, mes))
})

# Nombro los elementos por mes
names(resultados_por_mes) = paste0("mes_", meses_ini)

In [12]:
# Asigno nombre a cada grid con los leadtime
hindcast_0 = resultados_por_mes[["mes_07"]]
hindcast_1 = resultados_por_mes[["mes_06"]]
hindcast_2 = resultados_por_mes[["mes_05"]]
hindcast_3 = resultados_por_mes[["mes_04"]]

# Combinamos los grids en la dimensión temporal
hindcast_0_grid = bindGrid(hindcast_0, dimension = "time")
hindcast_1_grid = bindGrid(hindcast_1, dimension = "time")
hindcast_2_grid = bindGrid(hindcast_2, dimension = "time")
hindcast_3_grid = bindGrid(hindcast_3, dimension = "time")

## Cargar forecast hr (leadtime 0, 1, 2, 3)

In [13]:
anios = 2017:2022
meses_ini = c("07", "06", "05", "04")  # meses de inicialización

# Función generalizada para cargar los datos por mes y año
cargar_dato = function(anio, mes_ini) {

    yyyymm = paste0(anio, mes_ini)

    ruta = paste0(
        "/lustre/gmeteo/PTICLIMA/DATA/SEASONAL/",
        "seasonal-original-single-levels/medcof/forecast/hurs/ecmwf/51/", yyyymm, "/",
        "seasonal-original-single-levels_medcof_forecast_hurs_ecmwf_51_", yyyymm, ".ncml"
    )

    data_aux = loadGridData(dataset = ruta,
                            var = "hurs",
                            lonLim = lon,
                            latLim = lat,
                            season = c(7, 8, 9)) %>% suppressMessages %>% suppressWarnings

    return(data_aux)
}

# Creo una lista donde cada elemento es la salida para un mes de inicialización diferente
forecast = lapply(meses_ini, function(mes) {
    lapply(anios, function(anio) cargar_dato(anio, mes))
})

# Nombro los elementos por mes
names(forecast) = paste0("mes_", meses_ini)

In [14]:
# Asigno nombre a cada grid con los leadtime
forecast_0 = forecast[["mes_07"]]
forecast_1 = forecast[["mes_06"]]
forecast_2 = forecast[["mes_05"]]
forecast_3 = forecast[["mes_04"]]

# Combinamos los grids en la dimensión temporal
forecast_0_grid = bindGrid(forecast_0, dimension = "time")
forecast_1_grid = bindGrid(forecast_1, dimension = "time")
forecast_2_grid = bindGrid(forecast_2, dimension = "time")
forecast_3_grid = bindGrid(forecast_3, dimension = "time")

# Me quedo con los primeros 25 miembros
forecast_0_members = subsetGrid(forecast_0_grid, members = 1:25)
forecast_1_members = subsetGrid(forecast_1_grid, members = 1:25)
forecast_2_members = subsetGrid(forecast_2_grid, members = 1:25)
forecast_3_members = subsetGrid(forecast_3_grid, members = 1:25)

# Combinamos los grids de hindcast y forecast
ecmwf_0_grid = bindGrid(hindcast_0_grid, forecast_0_members, dimension = "time")
ecmwf_1_grid = bindGrid(hindcast_1_grid, forecast_1_members, dimension = "time")
ecmwf_2_grid = bindGrid(hindcast_2_grid, forecast_2_members, dimension = "time")
ecmwf_3_grid = bindGrid(hindcast_3_grid, forecast_3_members, dimension = "time")

## Cargar las observaciones ERA5-Land (punto de rocío)

In [15]:
# Define años y meses
anios = 1981:2022
meses = sprintf("%02d", c(7, 8, 9))

# Función para construir la ruta y cargar los datos
cargar_dato = function(anio, mes) {
    yyyy = paste0(anio)
    yyyymm = paste0(anio, mes)
    ruta = paste0(
        "/lustre/gmeteo/PTICLIMA/DATA/REANALYSIS/ERA5-Land/data/Iberia/day/d2m/", yyyy, "/",
        "d2m_ERA5-Land_", yyyymm, ".nc"
    )
    
    # Carga el dataset
    data_aux = loadGridData(dataset = ruta,
                            var = "d2m",
                            lonLim = lon,
                            latLim = lat) %>% suppressMessages %>% suppressWarnings

    return(data_aux)
}

# Crear combinaciones enero + febrero
combinaciones = do.call(rbind, lapply(anios, function(anio) {
  data.frame(
    anio = c(anio, anio, anio),
    mes  = c("07", "08", "09"),
    stringsAsFactors = FALSE
  )
}))

# Aplicar la función a cada combinación
d2m_era5_data = lapply(1:nrow(combinaciones), function(i) {
    cargar_dato(combinaciones$anio[i], combinaciones$mes[i])
})

In [16]:
# Combinamos los grids en la dimensión temporal
d2m_era5_time = bindGrid(d2m_era5_data, dimension = "time")

# Upscaling de la resolución de las observaciones
d2m_era5_ups = interpGrid(d2m_era5_time,
                          new.coordinates = getGrid(ecmwf_0_grid),
                          method = "bilinear") %>% suppressMessages %>% suppressWarnings

d2m_obs = subsetGrid(d2m_era5_ups, lonLim = c(-10, 5), latLim = c(35, 44))

# Pasamos las observaciones de Kelvin a Celsius
d2m_obs_cel = gridArithmetics(d2m_obs, 273.15, operator = "-")

In [17]:
# Recorto los datos a la región de España
hr_modelo_0 = subsetGrid(ecmwf_0_grid, lonLim = c(-10, 5), latLim = c(35, 44))
hr_modelo_1 = subsetGrid(ecmwf_1_grid, lonLim = c(-10, 5), latLim = c(35, 44))
hr_modelo_2 = subsetGrid(ecmwf_2_grid, lonLim = c(-10, 5), latLim = c(35, 44))
hr_modelo_3 = subsetGrid(ecmwf_3_grid, lonLim = c(-10, 5), latLim = c(35, 44))

## Calculo la humedad relativa de ERA-5 a patir del punto de rocío

In [18]:
# Presión de vapor de saturación: es(T) = 6.112 * exp(17.67*T / (T+243.5))
t2m_c = obs_cel

es_t = gridArithmetics(gridArithmetics(t2m_c, C = 17.67, operator = "*"), 
                      gridArithmetics(t2m_c, C = 243.5, operator = "+"), operator = "/")

es_t$Data = exp(es_t$Data)
es_t = gridArithmetics(es_t, C = 6.112, operator = "*")

# Presión de vapor real (con Td)
d2m_c = d2m_obs_cel

es_td = gridArithmetics(gridArithmetics(d2m_c, C = 17.67, operator = "*"), 
                       gridArithmetics(d2m_c, C = 243.5, operator = "+"), operator = "/")

es_td$Data = exp(es_td$Data)
es_td = gridArithmetics(es_td, C = 6.112, operator = "*")

# Humedad relativa
hr = gridArithmetics(es_td, es_t, operator = "/")
hr_obs = gridArithmetics(hr, C = 100, operator = "*")

In [19]:
# Guardo los datos
saveRDS(hr_modelo_0, file = "hr_botritis_model_vid_0.rds")
saveRDS(hr_modelo_1, file = "hr_botritis_model_vid_1.rds")
saveRDS(hr_modelo_2, file = "hr_botritis_model_vid_2.rds")
saveRDS(hr_modelo_3, file = "hr_botritis_model_vid_3.rds")

saveRDS(hr_obs, file = "hr_botritis_obs_vid.rds")